In [1]:
# using sqlite3 for data exploration + visualization. pandas - for df
import sqlite3
import pandas as pd

In [2]:
# ipython-sql for sql magic statements, matplotlib and seaborn for visualization 
!pip install ipython-sql
!pip install seaborn 
import seaborn as sns
%load_ext sql
import matplotlib.pyplot as plt

In [3]:
# this either creates a new database or uses the existing one if it exists. it's done in thee current directory
# here we make a new database - netflix.db
conn = sqlite3.connect('netflix.db')
cur = conn.cursor()

In [4]:
# check the current directory. That's where the database is stored
%pwd

'C:\\Users\\haykg\\Documents\\github\\Netflix_data_EDA'

In [5]:
# this is to be able to use the netflix database with magic sql commands
%sql sqlite:///netflix.db

In [6]:
df = pd.read_csv('netflix_titles.csv')

In [7]:
# pandas method that takes the pandas DataFrame df and writes it into my SQL database and names Chicago_socioeconomic_data.
# index = FALSE says **Do not write the DataFrame’s index** as a column in the SQL table. By default, `to_sql` will create 
# a column for the DataFrame’s index; setting `index=False` skips that.
# method = "multi" will allow to insert several rows simultaneously. Otherwise it would just be one row at a time.
df.to_sql("netflix", conn, if_exists="replace", index = False)

8807

In [8]:
# For below you can also
# Install the 'ipython-sql' and 'prettytable' libraries using pip
!pip install ipython-sql prettytable

# Import the 'prettytable' library, which is used to display data in a formatted table
import prettytable

# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'

In [10]:
%sql select * from netflix limit 3

 * sqlite:///netflix.db
Done.


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."


In [11]:
# checking the promary keys. We find there are no primary keys - so I will delete the useless "False" column and make show_id the main one.
%sql PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,show_id,TEXT,0,None,0
1,type,TEXT,0,None,0
2,title,TEXT,0,None,0
3,director,TEXT,0,None,0
4,cast,TEXT,0,None,0
5,country,TEXT,0,None,0
6,date_added,TEXT,0,None,0
7,release_year,INTEGER,0,None,0
8,rating,TEXT,0,None,0
9,duration,TEXT,0,None,0


In [12]:
# show_id is supposed to be the unique identifier - hence we need to check if there are any repeating values
%sql select show_id, count(*) from netflix group by show_id having count(*) > 1 limit 5

# no duplicates

 * sqlite:///netflix.db
Done.


show_id,count(*)


In [25]:
%%sql
--# Checking for nulls

with null_table as
(SELECT 'type' as column,         SUM(`type`       IS NULL) as null_count FROM netflix
  UNION ALL SELECT 'cast',         SUM(`cast`       IS NULL) FROM netflix
  UNION ALL SELECT 'show_id',      SUM(show_id      IS NULL) FROM netflix
  UNION ALL SELECT 'title',        SUM(title        IS NULL) FROM netflix
  UNION ALL SELECT 'director',     SUM(director     IS NULL) FROM netflix
  UNION ALL SELECT 'country',      SUM(country      IS NULL) FROM netflix
  UNION ALL SELECT 'date_added',   SUM(date_added   IS NULL) FROM netflix
  UNION ALL SELECT 'release_year', SUM(release_year IS NULL) FROM netflix
  UNION ALL SELECT 'rating',       SUM(rating       IS NULL) FROM netflix
  UNION ALL SELECT 'duration',     SUM(duration     IS NULL) FROM netflix
  UNION ALL SELECT 'listed_in',    SUM(listed_in    IS NULL) FROM netflix
  UNION ALL SELECT 'description',  SUM(description  IS NULL) FROM netflix)

select * from null_table where null_count > 0 order by null_count DESC



 * sqlite:///netflix.db
Done.


column,null_count
cast,825
date_added,10
rating,4
duration,3


In [17]:
%%sql
--# to find out if some directors are likely to work with some cast to update the director names where possible    

with directs(movie, direct_cast) AS
(select title, concat(director, '---', `cast`) from netflix)
select direct_cast, count(*) as instances from directs 
group by direct_cast having instances > 1 order by count(*) DESC limit 100              

 * sqlite:///netflix.db
Done.


direct_cast,instances
---,352
---David Attenborough,15
"Rajiv Chilaka---Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil",12
"---David Spade, London Hughes, Fortune Feimster",6
"---Michela Luci, Jamie Watson, Eric Peterson, Anna Claire Bartlam, Nicolas Aqui, Cory Doran, Julie Lemieux, Derek McGrath",5
"S.S. Rajamouli---Prabhas, Rana Daggubati, Anushka Shetty, Tamannaah Bhatia, Sathyaraj, Nassar, Ramya Krishnan, Sudeep",4
"Rathindran R Prasad---Aishwarya Rajesh, Vidhu, Surya Ganapathy, Madhuri, Pavel Navageethan, Avantika Vandanapu",4
Hernán Zin---,4
"---Michela Luci, Jamie Watson, Anna Claire Bartlam, Dante Zee, Eric Peterson",4
Walter C. Miller---Sam Kinison,3


In [18]:
%%sql
--# Let's look into the most common correspondances and check if there are cases with missing directors
--# then we will update the missing ones where the correspondance is sure
select title, director, `cast` from netflix where `cast` like '%Vatsal Dubey, Julie Tejwani, Rupa Bhimani%'

 * sqlite:///netflix.db
Done.


title,director,cast
Chhota Bheem,None,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem - Neeli Pahaadi,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Ganesh,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Krishna: Mayanagari,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Krishna: Pataliputra- City of the Dead,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Arun Shekher, Sanchit Wartak"
Chhota Bheem And The Broken Amulet,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem And The Crown of Valhalla,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem and the Incan Adventure,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem and The ShiNobi Secret,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem Aur Hanuman,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"


In [19]:
# one of those cases where exact same cast has the same director. so we can update a few nulls
%sql update netflix SET director ='Rajiv Chilaka' where `cast` like '%Vatsal Dubey, Julie Tejwani%' and director is null

 * sqlite:///netflix.db
2 rows affected.


[]

In [20]:
# the other cases of director - cast don't look connected so we update the other directors to "not known" below
%sql update netflix set director = 'Not Given' where director is null;
# to make sure no longer any null values for directors
%sql select count(*) from netflix where director is null

 * sqlite:///netflix.db
2632 rows affected.
 * sqlite:///netflix.db
Done.


count(*)
0


In [21]:
%sql PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,show_id,TEXT,0,None,0
1,type,TEXT,0,None,0
2,title,TEXT,0,None,0
3,director,TEXT,0,None,0
4,cast,TEXT,0,None,0
5,country,TEXT,0,None,0
6,date_added,TEXT,0,None,0
7,release_year,INTEGER,0,None,0
8,rating,TEXT,0,None,0
9,duration,TEXT,0,None,0


## cleaning nulls of countries
(i) We will check the directors, county couples with the number of times they used the country as a location. <br>
(ii) Then we will order the country_numbers for each director from 1 to ..., with 1 being the most common country for a dir. <br>
(iii) Then we will only choose the cases where director filmed in no1 country at least twice. <br>
(iv) Afterwards, we will update the nulls of countries corresponding to the directors that have a favorite country.

In [22]:
%%sql
--# below a table is created with the corresponding most often used countries
drop table if exists countries_to_update;
create table countries_to_update as
with counts_direct as (
  SELECT
    director,
    country,
    COUNT(*) AS country_count
  FROM
    netflix
  WHERE
    director IS NOT NULL AND country IS NOT NULL and director !='Not Given'
  GROUP BY
    director, country order by country_count desc),
    
ranked_counts as (
    select director, country, country_count, 
row_number() over(partition by director order by country_count) as country_rank 
from counts_direct order by country_count DESC
)

select director, country as most_used_country from ranked_counts where country_rank = 1 and country_count > 1

 * sqlite:///netflix.db
Done.
Done.


[]

In [23]:
%%sql
--# using the new table 'most_used_country' to update the countries in netflix
update netflix as n
set country = ctu.most_used_country 
from countries_to_update as ctu
where n.director = ctu.director and n.country is null

 * sqlite:///netflix.db
53 rows affected.


[]

In [24]:
%%sql
--# for the rest of the countries we put 'Not Given'
update netflix set country = "Not Given" where country is Null

 * sqlite:///netflix.db
778 rows affected.


[]

## dealing with the rest of the nulls
(i) As cast is irrelevant for the analysis, we will delete the column. <br>
(ii) Next we will drop the rows for 'delete_added', 'rating', 'duration' because those are very few and won't affect analysis.

In [34]:
%%sql 
--# remove the cast and description
alter table netflix drop `description`;
alter table netflix drop `cast`;

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: "`description`"
[SQL: --# remove the cast and description
alter table netflix drop `description`;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [35]:
%%sql
delete from netflix
where date_added is null or rating is null or duration is null

 * sqlite:///netflix.db
0 rows affected.


[]

In [36]:
%%sql
--# checking and seeing that there are no more nulls left
SELECT `type` as column,         SUM(`type`       IS NULL) as null_count FROM netflix
  UNION ALL SELECT 'show_id',      SUM(show_id      IS NULL) FROM netflix
  UNION ALL SELECT 'title',        SUM(title        IS NULL) FROM netflix
  UNION ALL SELECT 'director',     SUM(director     IS NULL) FROM netflix
  UNION ALL SELECT 'country',      SUM(country      IS NULL) FROM netflix
  UNION ALL SELECT 'date_added',   SUM(date_added   IS NULL) FROM netflix
  UNION ALL SELECT 'release_year', SUM(release_year IS NULL) FROM netflix
  UNION ALL SELECT 'rating',       SUM(rating       IS NULL) FROM netflix
  UNION ALL SELECT 'duration',     SUM(duration     IS NULL) FROM netflix
  UNION ALL SELECT 'listed_in',    SUM(listed_in    IS NULL) FROM netflix;

 * sqlite:///netflix.db
Done.


column,null_count
Movie,0
show_id,0
title,0
director,0
country,0
date_added,0
release_year,0
rating,0
duration,0
listed_in,0


## primary key added
No pk in the original table. We add it to the "id" column

In [39]:
%%sql
-- # adding a primary key to the table
drop table if exists netflix_old;
alter table netflix rename to netflix_old;

drop table if exists netflix;

CREATE TABLE netflix (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    show_id TEXT,
    type TEXT,
    title TEXT,
    director TEXT,
    country TEXT,
    date_added TEXT,
    release_year INTEGER,
    rating TEXT,
    duration TEXT,
    listed_in TEXT
);

insert into netflix(show_id, type, title, director, country, date_added, release_year, rating, duration, listed_in)
select show_id, type, title, director, country, date_added, release_year, rating, duration, listed_in from netflix_old

 * sqlite:///netflix.db
Done.
Done.
Done.
Done.
8790 rows affected.


[]

In [40]:
%%sql 
--# verifying the primary key
PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,show_id,TEXT,0,None,0
2,type,TEXT,0,None,0
3,title,TEXT,0,None,0
4,director,TEXT,0,None,0
5,country,TEXT,0,None,0
6,date_added,TEXT,0,None,0
7,release_year,INTEGER,0,None,0
8,rating,TEXT,0,None,0
9,duration,TEXT,0,None,0


## cleaning the "country" part
Below we make a many-to-many relationship between shows and countries. <br>
We make a table with just countries with their unique code first.

In [41]:
%%sql
--# Checking the number of movies shot in several countries
select count(country) from netflix where country like "%,%" ;

 * sqlite:///netflix.db
Done.


count(country)
1320


In [42]:
%%sql
--# checking for any other character to make sure that the countries are only seperated by a standard comma.
SELECT distinct(country)
FROM netflix where country glob '*[^A-Za-z0-9 ,]*';

 * sqlite:///netflix.db
Done.


country


In [43]:
%%sql
drop table if exists countries;
create table countries (id INTEGER PRIMARY KEY AUTOINCREMENT,
                       country varchar not null unique);

 * sqlite:///netflix.db
Done.
Done.


[]

In [44]:
%sql PRAGMA table_info(countries)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,country,varchar,1,None,0


In [114]:
%%sql
--# below I will make a new table with all the countries listed. I will use recursive formula for it.
--# the logic goes as follows - countries_table is a recursive temporary table - we first choose the first column to consist only
--# the first country by choosing only the characters up to the first ",". Second column are all the other countries after the first ","
--# Next we apply recursive formula where we select from this newly created temp. table countries_table with the same logic as decribed.
--# this makes a new table with the exact same structure. We "UNION ALL" both tables. This process of the second recursive table continues
--# until all second column components adre NULL. We also keep the ID from the original netflix table next to the countries.
--# This implies that the first col of the table are the movie id's which can be repeated if the film was shot in diff. countries.
--# and the single_country column shows the unique countries for each show
WITH recursive countries_table as (
    select id, 
    case 
        when instr(country, ',') > 0 then TRIM(substr(country,1,instr(country,',') -1))
        else TRIM(country) 
        end as single_country,
    case
        when instr(country, ',') > 0 then TRIM(substr(country, instr(country, ',') + 1))
        else null
        end as other_countries
    from netflix where country is not null and trim(country) <> ""

UNION ALL
    select id,
    case
        when instr(other_countries, ',') > 0 then TRIM(substr(other_countries,1,instr(other_countries,',') -1))
        else TRIM(other_countries) end,
    case
        when instr(other_countries, ',') > 0 then TRIM(substr(other_countries, instr(other_countries, ',') + 1))
        else null
        end
    from countries_table where other_countries is not null
)
select * from countries_table order by id limit 5;

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: country
[SQL: --# below I will make a new table with all the countries listed. I will use recursive formula for it.
--# the logic goes as follows - countries_table is a recursive temporary table - we first choose the first column to consist only
--# the first country by choosing only the characters up to the first ",". Second column are all the other countries after the first ","
--# Next we apply recursive formula where we select from this newly created temp. table countries_table with the same logic as decribed.
--# this makes a new table with the exact same structure. We "UNION ALL" both tables. This process of the second recursive table continues
--# until all second column components adre NULL. We also keep the ID from the original netflix table next to the countries.
--# This implies that the first col of the table are the movie id's which can be repeated if the film was shot in diff. countries.
--# and the single

In [58]:
%%sql
--## repeating the recursive code to get a new table with countries and show ids
drop table if exists shows_countries;
create table shows_countries_perm as 
with recursive new_countries as (
    select id,
    case
        when instr(country, ",") > 0 then TRIM(substr(country, 1, instr(country, ",") - 1))
        else trim(country)
        end as single_country,
    case
        when instr(country, ",") > 0 then TRIM(substr(country, instr(country, ",") + 1 ))
        else null
        end as other_countries
    from netflix where country is not null and trim(country) <> ""

    UNION ALL

    select id,
    case
        when instr(other_countries, ",") > 0 then TRIM(substr(other_countries, 1, instr(other_countries, ",") - 1))
        else trim(other_countries) end,
    case 
        when instr(other_countries, ",") > 0 then TRIM(substr(other_countries, instr(other_countries, ",") + 1))
        else null end
    from new_countries where other_countries is not null
)

select id, single_country as country from new_countries order by id;

 * sqlite:///netflix.db
Done.
(sqlite3.OperationalError) table shows_countries_perm already exists
[SQL: create table shows_countries_perm as 
with recursive new_countries as (
    select id,
    case
        when instr(country, ",") > 0 then TRIM(substr(country, 1, instr(country, ",") - 1))
        else trim(country)
        end as single_country,
    case
        when instr(country, ",") > 0 then TRIM(substr(country, instr(country, ",") + 1 ))
        else null
        end as other_countries
    from netflix where country is not null and trim(country) <> ""

    UNION ALL

    select id,
    case
        when instr(other_countries, ",") > 0 then TRIM(substr(other_countries, 1, instr(other_countries, ",") - 1))
        else trim(other_countries) end,
    case 
        when instr(other_countries, ",") > 0 then TRIM(substr(other_countries, instr(other_countries, ",") + 1))
        else null end
    from new_countries where other_countries is not null
)

select id, single_country as coun

In [59]:
%%sql
--# I checked manually the similar names for the same country but none were found.
--# Now creating a table with distinct countries
drop table if exists countries_table;
create table countries_table (id integer primary key autoincrement,
country varchar);
insert into countries_table(country)
select distinct(country) from shows_countries_perm where lower(country) not like "%not given%" order by country;

 * sqlite:///netflix.db
Done.
Done.
123 rows affected.


[]

In [60]:
%%sql
select * from countries_table limit 20

 * sqlite:///netflix.db
Done.


id,country
1,
2,Afghanistan
3,Albania
4,Algeria
5,Angola
6,Argentina
7,Armenia
8,Australia
9,Austria
10,Azerbaijan


In [61]:
%%sql
--# here we create the table shows_countries, which keeps the many to many relationship between the countries and shows
create table if not exists shows_countries as 
select s.id as show_id, c.id as country_id from shows_countries_perm as s, countries_table as c
where s.country = c.country limit 10;

 * sqlite:///netflix.db
Done.


[]

In [62]:
%%sql
--# dropping the country column from netflix
alter table netflix drop column country

 * sqlite:///netflix.db
Done.


[]

In [63]:
%%sql
--# let's recap what tables we have so far
--# the netflix table
select * from netflix limit 5;
--# the countries table
select * from countries_table limit 5;
--# the shows_countries table that holds the relationship - shows - countries
select * from shows_countries limit 5;

 * sqlite:///netflix.db
Done.
Done.
Done.


show_id,country_id
1,117
2,101
5,47
6,117
8,117


In [64]:
%%sql
select * from netflix limit 5;

 * sqlite:///netflix.db
Done.


id,show_id,type,title,director,date_added,release_year,rating,duration,listed_in
1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"September 25, 2021",2020,PG-13,90 min,Documentaries
2,s2,TV Show,Blood & Water,Not Given,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
3,s3,TV Show,Ganglands,Julien Leclercq,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure"
4,s4,TV Show,Jailbirds New Orleans,Not Given,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV"
5,s5,TV Show,Kota Factory,Not Given,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies"


## altering the format of date_added 
Changing the date_added column to have the correct - date type


In [65]:
# using python pandas to change the date type to datetime
df1 = pd.read_sql_query('SELECT id, trim(date_added) as date_added FROM netflix', conn)
df2 = pd.to_datetime(df1["date_added"])

In [72]:
# creating a netflix_date table that will only have the dates in order as the column
df2.dt.date.to_sql('netflix_date', conn, if_exists = 'replace', index = False)

8790

In [73]:
%sql select * from netflix_date limit 20

 * sqlite:///netflix.db
Done.


date_added
2021-09-25
2021-09-24
2021-09-24
2021-09-24
2021-09-24
2021-09-24
2021-09-24
2021-09-24
2021-09-24
2021-09-24


In [74]:
%%sql
Select count(*) from netflix 

 * sqlite:///netflix.db
Done.


count(*)
8790


In [80]:
%%sql 
PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,show_id,TEXT,0,None,0
2,type,TEXT,0,None,0
3,title,TEXT,0,None,0
4,director,TEXT,0,None,0
5,date_added,TEXT,0,None,0
6,release_year,INTEGER,0,None,0
7,rating,TEXT,0,None,0
8,duration,TEXT,0,None,0
9,listed_in,TEXT,0,None,0


In [79]:
%%sql
--# adding a new DATE type column to keep the newly formatted date
alter table netflix add column new_date DATE;

 * sqlite:///netflix.db
(sqlite3.OperationalError) duplicate column name: new_date
[SQL: --# adding a 
alter table netflix add column new_date DATE;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [83]:
%%sql 
update netflix
set new_date = (select date_added from netflix_date);
alter table netflix
drop column date_added;
alter table netflix
rename column new_date to date_added;
select * from netflix limit 20

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: new_date
[SQL: update netflix
set new_date = (select date_added from netflix_date);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Updating duration
Let's change duration so it is consistent in formating.

Step by step:
(i) verify there are only 2 types of "movie" and "TV show". If not make the evident ones same name exactly.
(ii) check similar forms for min, mins - maybe there are hours or diff then season
(iii) next alter the netflix table and add new columns - uration_movie_minutes and duration_TV_seasons - each must contain only the int duration of either type - minutes for movies and seasons for TV shows
(iv) drop the old duration column

In [158]:
%%sql
select * from netflix limit 5;

 * sqlite:///netflix.db
Done.


id,show_id,type,title,director,release_year,rating,listed_in,date_added,duration_movie_minutes,duration_TV_seasons
1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,PG-13,Documentaries,2021-09-25,90,None
2,s2,TV Show,Blood & Water,Not Given,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2021-09-25,None,2
3,s3,TV Show,Ganglands,Julien Leclercq,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Action & Adventure",2021-09-25,None,1
4,s4,TV Show,Jailbirds New Orleans,Not Given,2021,TV-MA,"Docuseries, Reality TV",2021-09-25,None,1
5,s5,TV Show,Kota Factory,Not Given,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV Comedies",2021-09-25,None,2


In [109]:
%%sql
select count(*) from netflix where type = "TV Show" and duration glob "*Season*";

 * sqlite:///netflix.db
Done.


count(*)
2664


In [113]:
%%sql
alter table netflix
drop column TV_duration;

 * sqlite:///netflix.db
Done.


[]

In [135]:
%%sql
drop table if exists durations;
create table durations as
with duration_table as 
(select id, type, 
cast(trim(substr(duration, 1, instr(duration, ' '))) as integer) as duration from netflix)
select * from duration_table order by id asc;

 * sqlite:///netflix.db
Done.
Done.


[]

In [138]:
%sql select * from durations limit 5;

 * sqlite:///netflix.db
Done.


id,type,duration
1,Movie,90
2,TV Show,2
3,TV Show,1
4,TV Show,1
5,TV Show,2


In [140]:
%%sql
alter table netflix
add column duration_movie_minutes integer;

 * sqlite:///netflix.db
(sqlite3.OperationalError) duplicate column name: duration_movie_minutes
[SQL: alter table netflix
add column duration_movie_minutes integer;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [141]:
%%sql
alter table netflix
add column duration_TV_seasons integer;

 * sqlite:///netflix.db
Done.


[]

In [142]:
%%sql
update netflix
set duration_movie_minutes = CASE
    WHEN type = "TV Show" then NULL
    ELSE (SELECT duration from durations where durations.id = netflix.id) end;

 * sqlite:///netflix.db
8790 rows affected.


[]

In [144]:
%%sql
update netflix
set duration_TV_seasons = case
    when type = "Movie" then Null
    else (select duration from durations where durations.id = netflix.id) end;

 * sqlite:///netflix.db
8790 rows affected.


[]

In [154]:
%%sql
alter table netflix drop column duration;

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: "duration"
[SQL: alter table netflix drop column duration;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [157]:
%%sql
--# no similar forms of the same rating - all different
select distinct(rating) from netflix;

 * sqlite:///netflix.db
Done.


rating
PG-13
TV-MA
PG
TV-14
TV-PG
TV-Y
TV-Y7
R
TV-G
G


## dropping the listed_in
As the purpos of general analysis, we don't need the genres, hence we drop the listed_in column

In [159]:
%%sql
alter table netflix drop column listed_in;

 * sqlite:///netflix.db
Done.


[]

In [160]:
%%sql
select * from netflix limit 5;

 * sqlite:///netflix.db
Done.


id,show_id,type,title,director,release_year,rating,date_added,duration_movie_minutes,duration_TV_seasons
1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,2020,PG-13,2021-09-25,90,None
2,s2,TV Show,Blood & Water,Not Given,2021,TV-MA,2021-09-25,None,2
3,s3,TV Show,Ganglands,Julien Leclercq,2021,TV-MA,2021-09-25,None,1
4,s4,TV Show,Jailbirds New Orleans,Not Given,2021,TV-MA,2021-09-25,None,1
5,s5,TV Show,Kota Factory,Not Given,2021,TV-MA,2021-09-25,None,2
